In [1]:
using GpABC

┌ Info: Precompiling GpABC [e850a1a4-d859-11e8-3d54-a195e6d045d3]
└ @ Base loading.jl:1317


In [2]:
include("run_remotely.jl")

run_batch (generic function with 1 method)

In [3]:
function simulate(params)
    pdict = Dict(
        "p-drop-contact" => params[1], 
#        "speed-expl-stay" => params[2], 
#        "speed-expl-move" => params[3], 
        "p-info-contacts" => params[2], 
        "p-transfer-info" => params[3])
#        "error" => params[6], 
#        "error-risk" => params[7], 
#        "error-frict" => params[8], 
#        "risk-scale" => params[9], 
#        "p-notice-death-c" => params[10], 
#        "p-notice-death-o" => params[11], 
#        "speed-risk-indir" => params[12], 
#        "speed-risk-obs" => params[13], 
#        "speed-expl-risk" => params[14], 
#        "path-penalty-risk" => params[15])
    args = " -m ../map_med1.json -s ../mediterranean '--wait 10 --warmup 100 --int 0.01 0.077 0.153 0.428 0.389 --risks 0.01 0.023 0.020 0.029 0.048'  -t 500"
    for (k, v) in pdict
        args *= " --$k $v" 
    end
    
    args *= " --rand-seed-sim $(rand(1:100000)) --rand-seed-world $(rand(1:100000))"
    
    xdirs = [readdir("incoming"); readdir("running"); readdir("done")]
    filter!(x -> startswith(x, "x"), xdirs)
    mx = 1
    for dir in xdirs
        m = match(r"x([0-9]+)_", dir)
        if m == nothing 
            continue
        end
        n = parse(Int, m.captures[1])
        if mx <= n
            mx = n+1
        end
    end
    
    prefix = "x$(mx)_$(Threads.threadid())_"
    server = "sotonhpc"
    dir = "Science/southampon/runs/mediterranean"
    
    results = run_batch(server, dir, prefix, [args])[1]

    result
end

function costs(result)
    arrivals = [0.657941144053786, 0.207848228494849, 0.599476123312513]
    mort = [0.022736973763885, 0.019768432868398, 0.028785488958991, 0.047819332348149]
    interc = [1.43020016842651, 0.884092762025546, 0.525283057533685]

    d1 = sqrt(sum((arrivals .- results[1]) .^ 2))
    d2 = sqrt(sum((mort .- results[2]) .^ 2))
    d3 = sqrt(sum((interc .- results[3]) .^ 2))
    
    d1 + d2 * 10 + d3
end
    

costs (generic function with 1 method)

In [5]:
using Distributions

In [6]:
priors = [Uniform(0, 1), Uniform(0, 1), Uniform(0, 1)]

3-element Vector{Uniform{Float64}}:
 Uniform{Float64}(a=0.0, b=1.0)
 Uniform{Float64}(a=0.0, b=1.0)
 Uniform{Float64}(a=0.0, b=1.0)